In [1]:
import pandas as pd
import networkx as nx
from collections import deque
from pyvis.network import Network
from tqdm import tqdm
from lib.store import Store
from collections import Counter
from typing import Dict
from sklearn.preprocessing import MinMaxScaler
import random
factor = 150
data_dir = "data/dataset"
save_path = "data/samples4"
partitions_dir = "partitions.parquet"
splits = ['train', 'val', 'test']

from lib.naming_corrections import (
    FEATURE_COLUMNS_OTHERS,
    FEATURES_NAMES_FROM_NEW_CACHE,
    FEATURES_NAMES_FROM_PRELOADED_CACHE,
    TABLES_COLUMNS_DEFAULT_LEGACY,
    TABLES_V5_2_V4_RENAME_LEGACY,
)

# --------------------------------------------------------------
# DEFINITIONS
# --------------------------------------------------------------


FEATURE_COLUMNS = FEATURES_NAMES_FROM_PRELOADED_CACHE + FEATURE_COLUMNS_OTHERS
TABLES_V5_2_V4_RENAME = TABLES_V5_2_V4_RENAME_LEGACY
TABLES_COLUMNS_DEFAULT = TABLES_COLUMNS_DEFAULT_LEGACY

In [18]:
def generic_bfs_edges(G, source, label_dict, neighbors=None, depth_limit=None, sort_neighbors=None, factor=100):
    """Iterate over edges in a breadth-first search.

    The breadth-first search begins at `source` and enqueues the
    neighbors of newly visited nodes specified by the `neighbors`
    function.

    Parameters
    ----------
    G : NetworkX graph

    source : node
        Starting node for the breadth-first search; this function
        iterates over only those edges in the component reachable from
        this node.

    neighbors : function
        A function that takes a newly visited node of the graph as input
        and returns an *iterator* (not just a list) of nodes that are
        neighbors of that node. If not specified, this is just the
        ``G.neighbors`` method, but in general it can be any function
        that returns an iterator over some or all of the neighbors of a
        given node, in any order.

    depth_limit : int, optional(default=len(G))
        Specify the maximum search depth

    sort_neighbors : function
        A function that takes the list of neighbors of given node as input, and
        returns an *iterator* over these neighbors but with custom ordering.

    Yields
    ------
    edge
        Edges in the breadth-first search starting from `source`.

    Examples
    --------
    >>> G = nx.path_graph(3)
    >>> print(list(nx.bfs_edges(G, 0)))
    [(0, 1), (1, 2)]
    >>> print(list(nx.bfs_edges(G, source=0, depth_limit=1)))
    [(0, 1)]

    Notes
    -----
    This implementation is from `PADS`_, which was in the public domain
    when it was first accessed in July, 2004.  The modifications
    to allow depth limits are based on the Wikipedia article
    "`Depth-limited-search`_".

    .. _PADS: http://www.ics.uci.edu/~eppstein/PADS/BFS.py
    .. _Depth-limited-search: https://en.wikipedia.org/wiki/Depth-limited_search
    """
    if callable(sort_neighbors):
        _neighbors = neighbors
        neighbors = lambda node: iter(sort_neighbors(_neighbors(node)))
    
    ratio = {0: 0, 1: 0} # counter for class 0 and class 1
    visited = {source} # set
    if depth_limit is None:
        depth_limit = len(G)
    # neighbors(source): return a generator of source's neighbors
    queue = deque([(source, depth_limit, neighbors(source))])
    ratio[label_dict[source]] += 1
    
    edges = []
    
    while queue:
        #i = random.choice(range(len(queue)))
        #parent, depth_now, children = queue[i]
        parent, depth_now, children = queue[0] # BFS: queue[0]
        try:
            child = next(children)
            if child not in visited:
                edges.append((parent, child))
                visited.add(child)
                ratio[label_dict[child]] += 1
                if depth_now > 1:
                    queue.append((child, depth_now - 1, neighbors(child)))
            try:
                if ratio[0]/ratio[1] >= factor:
                    #print('factor is fulfiled:', ratio)
                    return edges
            except ZeroDivisionError as e:
                pass
        except StopIteration:
            #del queue[i]
            queue.popleft()
    
    #print('factor is not fulfiled but the traverse of weekly connected component is finished:', ratio)
    return edges

In [8]:
def load_local_data_store(data_dir:str) -> Store:   
    # build the store
    store = Store(
        base_dir=data_dir,
        protocol='file'
    )
    return store

datastore = load_local_data_store(data_dir)
df_p = pd.read_parquet(
        datastore.open_file(partitions_dir)
    ).reset_index(drop=True).reset_index()

In [9]:
data_dir

'data/dataset'

In [10]:
counters = {}
graph_data = {}
labelled = {}
others = {}
for sp, A in df_p.groupby('split'):
    graph_data[sp] = {
        x: None
        for x in sorted(A['index'])
    }
    labelled[sp] = {
        x: None
        for x in sorted(A['index'])
    }
    counters[sp] = Counter()

In [11]:
graph_data

{'test': {424: None,
  425: None,
  426: None,
  427: None,
  428: None,
  429: None,
  430: None,
  431: None,
  432: None,
  433: None,
  434: None,
  435: None,
  436: None,
  437: None,
  438: None,
  439: None,
  440: None,
  441: None,
  442: None,
  443: None,
  444: None,
  445: None,
  446: None,
  447: None,
  448: None,
  449: None,
  450: None,
  451: None,
  452: None,
  453: None,
  454: None,
  455: None,
  456: None,
  457: None,
  458: None,
  459: None,
  460: None,
  461: None,
  462: None,
  463: None,
  464: None,
  465: None,
  466: None,
  467: None,
  468: None,
  469: None,
  470: None},
 'train': {0: None,
  1: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None,
  12: None,
  13: None,
  14: None,
  15: None,
  16: None,
  17: None,
  18: None,
  19: None,
  20: None,
  21: None,
  22: None,
  23: None,
  24: None,
  25: None,
  26: None,
  27: None,
  28: None,
  29: None,
  30: None,
  31: Non

In [12]:
def fit_scaler(graph_data: Dict):
    print("Fitting Scaler...")
    scaler = MinMaxScaler()
    for p in tqdm(graph_data):
        df_f = pd.read_parquet(f"./data/dataset/cache/features/features_{p}.parquet")

        X = df_f[FEATURE_COLUMNS].fillna(value=0.0).values
        scaler.partial_fit(X)
    return scaler

scaler = fit_scaler(graph_data['train'])

  0%|          | 0/377 [00:00<?, ?it/s]

Fitting Scaler...


100%|██████████| 377/377 [00:52<00:00,  7.17it/s]


In [19]:
p_dict = {'index': [], 'split': []}

idx = 0
#for sp in splits:
for p in tqdm(graph_data['test']):
    
    network = pd.read_parquet(f'./{data_dir}/cache/edges/edges_{p}.parquet')
    labels = pd.read_parquet(f'./{data_dir}/labels/labels_{p}.parquet')
    features = pd.read_parquet(f'./{data_dir}/cache/features/features_{p}.parquet')
    G = nx.from_pandas_edgelist(network, 'from', 'to')
    successors = G.neighbors # can retrieve all neighbors of a particular node with []
    # replace all label 2 as label 0
    labels.loc[labels['label'] == 2, 'label'] = 0
    label_dict = dict(zip(labels.node, labels.label))
    node_set = set()
    #print(len(labels[labels['label']==1].node.values))
    for _, pos_node in enumerate(labels[labels['label']==1].node.values):
        try: # the node in the label parquet may not exist in the edge parquet
            result = generic_bfs_edges(G, pos_node, label_dict, successors, factor=factor)
        except Exception as e:
            continue
        
        df_result = pd.DataFrame(result, columns=['from', 'to'])
        #node_set.update(df_result.values.reshape(-1).tolist())
        
        # undirected to directed
        directed1 = network.merge(df_result, how='inner', left_on=['from', 'to'], right_on=['from', 'to'])
        directed2 = network.merge(df_result, how='inner', left_on=['from', 'to'], right_on=['to', 'from'])[['from_x', 'to_x', 'partition']].rename(columns={"from_x": "from", "to_x": "to", 'partition': 'partition'})
        samples = pd.concat([directed1, directed2], axis=0)
        edges_list = samples[['from', 'to']].values
        
        df_node = pd.DataFrame(set(edges_list.reshape(-1)), columns=['node'])
        #if len(df_node) > 10:
            #node_set.update(df_node.values.reshape(-1).tolist())
            
        sample_labels = labels.merge(df_node, how='inner').sort_values(by=['node'])
        sample_features = features.merge(sample_labels, how='inner', on='txid').sort_values(by=['node']).drop(['node'], axis=1)
        mapping = dict(zip(sample_labels.node.values, range(len(sample_labels))))
        samples[['from', 'to']] = samples[['from', 'to']].replace(mapping)
        sample_labels.node = [i for i in range(len(sample_labels))]
        #sample_features[FEATURE_COLUMNS] = scaler.transform(sample_features[FEATURE_COLUMNS].values) # normalize
        samples.to_parquet(f'./{save_path}/cache/edges/edges_{idx}.parquet')
        sample_labels.to_parquet(f'./{save_path}/labels/labels_{idx}.parquet')
        sample_features.to_parquet(f'./{save_path}/cache/features/features_{idx}.parquet')

        p_dict['index'].append(idx)
        p_dict['split'].append(sp)
        #idx += 1
        #else:
        #print("Subgraph is too small. Skip.")
        #continue

        # visualization
        SG = nx.Graph(create_using=nx.DiGraph)
        for edge in edges_list:
            if label_dict[edge[0]]: 
                SG.add_node(int(edge[0]), size=15, group=1)
            else:
                SG.add_node(int(edge[0]), size=15, group=2)

            if label_dict[edge[1]]: 
                SG.add_node(int(edge[1]), size=15, group=1)
            else:
                SG.add_node(int(edge[1]), size=15, group=2)
            SG.add_edge(
                        int(edge[0]),
                        int(edge[1])
                    )

        net = Network(notebook=True, directed =True, font_color="black")
        #net.show_buttons(filter_=['physics
        net.set_options("""var options = {"edges": {"smooth": false}}""")
        net.from_nx(SG)
        
        net.show(f"./html/example_{idx}.html")
        idx += 1
    #break
    #sample_nodes = pd.DataFrame(sorted(node_set), columns=['node'])
    #display(sample_nodes)
    #print(sample_nodes.isin(labels[labels['label']==1].node.values).sum())"""
    
    #sample_nodes.to_parquet(f'./{save_path}/cache/masks/masks_{p}.parquet', index=False)
#pd.DataFrame.from_dict(p_dict).to_parquet(f"./{save_path}/partitions.parquet")

  0%|          | 0/47 [00:00<?, ?it/s]

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


  2%|▏         | 1/47 [00:01<01:31,  1.99s/it]

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Loca

  4%|▍         | 2/47 [00:04<01:37,  2.16s/it]

...

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Loca

 98%|█████████▊| 46/47 [02:53<00:04,  4.20s/it]

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


100%|██████████| 47/47 [02:54<00:00,  3.72s/it]

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [41]:
pd.DataFrame.from_dict({'index': range(20), 'split': ['test']*20}).to_parquet("partitions.parquet")

In [42]:
pd.read_parquet("partitions.parquet")

,index,split
0,0,test
1,1,test
2,2,test
3,3,test
4,4,test
5,5,test
6,6,test
7,7,test
8,8,test
9,9,test


In [18]:
from pickle import dump, load
#dump(scaler, open('scaler.pkl', 'wb'))

# And now to load...

scaler = load(open('scaler.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'scaler.pkl'

In [ ]:
# visualization
"""SG = nx.Graph(create_using=nx.DiGraph)
for edge in edges_list:
    if label_dict[edge[0]]: 
        SG.add_node(int(edge[0]), size=15, group=1)
    else:
        SG.add_node(int(edge[0]), size=15, group=2)

    if label_dict[edge[1]]: 
        SG.add_node(int(edge[1]), size=15, group=1)
    else:
        SG.add_node(int(edge[1]), size=15, group=2)
    SG.add_edge(
                int(edge[0]),
                int(edge[1])
            )

net = Network(notebook=True, directed =True, font_color="black")
#net.show_buttons(filter_=['physics'])
net.set_options("""
"""var options = {"edges": {"smooth": false}}"""
""")
net.from_nx(SG)

net.show(f"./html/example_{idx}.html")"""

In [9]:
data_dir

'data/samples3'

In [6]:

for sp in splits:
    num_n = 0
    num_e = 0
    num_y = 0
    for p in tqdm(graph_data[sp]):
        network = pd.read_parquet(f'./{data_dir}/cache/edges/edges_{p}.parquet')
        labels = pd.read_parquet(f'./{data_dir}/labels/labels_{p}.parquet')
        features = pd.read_parquet(f'./{data_dir}/cache/features/features_{p}.parquet')
        nodes = labels[['txid', 'label']].drop_duplicates()
        edges = network[['from', 'to']].drop_duplicates()
        num_e += len(edges)
        num_n += len(nodes)
        num_y += len(nodes[nodes['label'] == 1])
    print(sp,":")
    print(num_n, num_e, num_y)

KeyError: 'train'

In [17]:
graph_data

{}

In [1]:
import pandas as pd

In [4]:
a = pd.read_parquet("./data/dataset/partitions.parquet")

In [10]:
a['time_start'][1] - a['time_start'][0] 

31555.0

In [14]:
pd.read_parquet("./data/dataset/cache/features/features_0.parquet")

,txid,num_sent,ave_amt_in,tot_amt_sent,sd_amt_sent,out_deg,ave_amt_out,tot_amt_recv,sd_amt_recv,ave_neigh_in_in,...,from_count_mean,from_count_std,from_count_sum,from_count_max,from_count_min,to_count_mean,to_count_std,to_count_sum,to_count_max,to_count_min
0,cee8b8dd41fbd537039e0beb06411bd6c1eb2db792587b...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2894a2cc3f17c2ec8ee3a9d2b27012e357d77b7e128ef0...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,d41347534dce3e8ffcffff455f0838d6c7d4360868ab79...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,992a18aec3db344be900e75b014168e060b5d0335a9108...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38ff1b8ecc0c0646a0a25a76c4b4b0dc3abc154c33e01e...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101612,199d9f66785fedbe88bc139f19d36352ac692b73b3f9c5...,1.0,0.000384,0.000384,0.0,2.0,0.000190,0.000381,0.000269,0.0,...,8.0,0.0,8.0,8.0,8.0,9.0,0.0,9.0,9.0,9.0
101613,cd5b5981258712d4e37dcdbe2d0cc67ffeec82e2034a27...,1.0,0.000381,0.000381,0.0,2.0,0.000189,0.000378,0.000268,0.0,...,8.0,0.0,8.0,8.0,8.0,9.0,0.0,9.0,9.0,9.0
101614,d35bbb015bbe63f752a65dcc69e0b893d6ad221291de29...,1.0,0.000378,0.000378,0.0,2.0,0.000188,0.000376,0.000266,0.0,...,8.0,0.0,8.0,8.0,8.0,9.0,0.0,9.0,9.0,9.0
101615,bd213359db030d57ea2b743922c6d6cee4883e0b0735fe...,1.0,0.000376,0.000376,0.0,2.0,0.000187,0.000373,0.000264,0.0,...,8.0,0.0,8.0,8.0,8.0,9.0,0.0,9.0,9.0,9.0


In [12]:
import time

In [13]:
time.time()

1681179580.342974